In [ ]:
# Imports and configuration
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

%load_ext line_profiler
%config InlineBackend.figure_format = 'retina'
sns.set_style("ticks")
plt.rc("axes.spines", top=False, right=False)

In [ ]:
# Model Parameters
import model

num_agents = 200
ticks = 2000
match_on_order = True

def run_sim(progress_bar = True):
    """Run and return a havven simulation"""
    m = model.Havven(num_agents, match_on_order=match_on_order)
    
    if progress_bar:
        for _ in tqdm(range(ticks)):
            m.step()
    else:
        for _ in range(ticks):
            m.step()
    
    return m

In [ ]:
# Run the model here
havven = run_sim()

In [ ]:
# Collect and plot the data

df = havven.datacollector.get_model_vars_dataframe()
df[["Nomin Price"]].plot()
df[["Curit Price"]].plot()
df[["Curit/Nomin Price"]].plot()
df[["Curit Demand", "Curit Supply"]].plot()
df[["Nomin Demand", "Nomin Supply"]].plot()
df[["Fiat Demand", "Fiat Supply"]].plot()
df[["Profit %"]].plot()
df[["Escrowed Curits", "Nomins"]].plot()
df[["Fee Pool", "Fees Distributed"]].plot()

plt.show()

In [ ]:
# Profiling
# import agents, orderbook

# Overall sim
# Result: 99.8% time spent in m.step()
# %lprun -f run_sim run_sim(False)

# Individual sim steps
# Result: 83% in agent.step(); 17% in datacollector.collect()
# %lprun -f model.Havven.step run_sim(False)

# Per-agent sim steps
# Result: ~40% each in agents.MarketPlayer.sell_fiat_for_curits and sell_nomins_for_curits,
#         8% in OrderBook.Order.cancel
# %lprun -f agents.Banker.step run_sim(False)

# Market player trades
# Result: 92% in OrderBook.buy, 6% in OrderBook.lowest_ask_price
# %lprun -f agents.MarketPlayer.sell_fiat_for_curits run_sim(False)
# %lprun -f agents.MarketPlayer.sell_nomins_for_curits run_sim(False)

# Order cancellations
# Result: 74% in OrderBook.buy_orders.remove, 8% in Orderbook.step,
#         6% in agents.MarketPlayer.notify_cancelled, 5% in agents.MarketPlayer.orders.remove
# %lprun -f orderbook.Bid.cancel run_sim(False)
# %lprun -f orderbook.Ask.cancel run_sim(False)

# Orderbook buy and ask prices
# Result: 93% orderbook.OrderBook.bid, 6% orderbook.OrderBook.lowest_ask_price
# %lprun -f orderbook.OrderBook.buy run_sim(False)

# Orderbook bid
# Result: 79% orderbook.OrderBook.match, 20% construction of Bids
# %lprun -f orderbook.OrderBook.bid run_sim(False)

# Matching functions
# Results: 24% match loop condition, 10% grabbing prev asks and bids from order lists
#          32% matcher functions, 16% spread calculation, 15% recalculating prices.
# %lprun -f orderbook.OrderBook.match run_sim(False)

# Bid construction
# Result: 22% superconstructor, 5% issuer order set addition,
#         59% order book buy order addition, 9% book.step
# %lprun -f orderbook.Bid.__init__ run_sim(False)